In [1]:
%%capture

!pip install "langchain==0.2.11"
!pip install "langchain-core==0.2.43"
!pip install langchain-google-genai

# Importing libraries

In [20]:
# Import Basic libraries
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

# Formatting
from IPython.display import Markdown, display

# Import Google Generative AI
from langchain_google_genai import ChatGoogleGenerativeAI 

# LangChain Components
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

# usually we start loading with environment variables
load_dotenv(override = True)

True

# API Initialization

In [4]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [15]:
def llm_model(prompt_txt, params=None):

    model_id = "gemini-2.0-flash"

    default_params = {
        "temperature" : 0,
        "max_tokens" : None, 
        "timeout" : None, 
        "max_retries" : 2
    }

    if params:
        default_params.update(params)

    # Create LLM directly
    llm = ChatGoogleGenerativeAI(
        model = model_id, 
        temperature = default_params["temperature"], 
        max_tokens = default_params["max_tokens"], 
        timeout = default_params["timeout"], 
        max_retries = default_params["max_retries"]
    )

    response = llm.invoke(prompt_txt)

    return response

In [14]:
ai_msg = llm_model(prompt_txt = [("system", "You are a helpful assistant that translates English to French. Translate the user sentense"), 
    ("human", "I love programming")])
print(ai_msg)    

Unexpected argument 'params' provided to ChatGoogleGenerativeAI.


content="J'adore la programmation." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--3d09e71f-8138-4d60-91b5-f8d68b74b030-0' usage_metadata={'input_tokens': 19, 'output_tokens': 7, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


# Prompt Engineering 

#### 1. Basic Prompt

A basic prompt is the simplest form of prompting, where you provide a short text or phrase to the model without any special formatting or instructions. The model generates a continuation based on patterns it has learned during training. Basic prompts are useful for exploring the model's capabilities and understanding how it naturally responds to minimal input

In [23]:
params = {
    "max_tokens" : 128, 
    "temperature" : 0.5, 
    "max_retries" : 2
}

prompt = "The wind is "

response = llm_model(prompt, params)
print(f"prompt : {prompt} \n")
print(f"response : {display(Markdown(response.content))} \n")

prompt : The wind is  



The wind is **blowing**.

response : None 



In [24]:
prompts = [
    "The future of artificial intellgence is",
    "Once upon a time in a distant galaxy",
    "The benefits of sustainable energy include"
]

for prompt in prompts:
    response = llm_model(prompt, params)
    print(f"prompt : {prompt} \n")
    print(f"response : {display(Markdown(response.content))} \n")

prompt : The future of artificial intellgence is 



The future of artificial intelligence is a topic of much discussion and speculation, with potential impacts across nearly every aspect of human life. Here's a breakdown of some key areas:

**Potential Positive Impacts:**

*   **Automation and Efficiency:** AI can automate repetitive tasks, freeing up humans for more creative and strategic work. This could lead to increased productivity and economic growth.
*   **Healthcare Advancements:** AI can assist in diagnosis, drug discovery, personalized medicine, and robotic surgery, potentially leading to better patient outcomes and more efficient healthcare systems.
*   **Improved Decision-Making:** AI algorithms can analyze vast amounts of data

response : None 

prompt : Once upon a time in a distant galaxy 



Once upon a time in a distant galaxy, far, far beyond the swirling nebulae of known space, existed a planet called Xylos. Xylos wasn't a typical planet of verdant forests or shimmering oceans. Instead, it was a world sculpted entirely from crystal. Towering spires of amethyst pierced the cerulean sky, while canyons of glittering quartz snaked across the landscape. The very air hummed with a low, resonant frequency, a song of the crystals themselves.

The inhabitants of Xylos were the Luminians, beings of pure light and energy, housed within crystalline shells. They were not born

response : None 

prompt : The benefits of sustainable energy include 



The benefits of sustainable energy are numerous and span environmental, economic, and social aspects. Here's a breakdown:

**Environmental Benefits:**

*   **Reduced Greenhouse Gas Emissions:** This is the most significant benefit. Sustainable energy sources like solar, wind, hydro, and geothermal produce little to no greenhouse gases during operation, mitigating climate change.
*   **Improved Air Quality:**  Unlike fossil fuels, sustainable energy sources don't release harmful air pollutants like sulfur dioxide, nitrogen oxides, and particulate matter, leading to cleaner air and reduced respiratory problems.
*   **Water Conservation:** Many sustainable energy technologies, particularly solar and wind

response : None 



#### 2. Zero-Shot Prompts

Zero-shot prompting is a technique where the model performs a task without any examples or prior specific training. This approach tests the model's ability to understand instructions and apply its knowledge to a new context without demonstration. Zero-shot prompts typically include clear instructions about what the model should do, allowing it to leverage its pre-trained knowledge effectively.

In [26]:
prompt = """Classify the following sentence as true or false:
                'The Eiffel Tower is located in Berlin.'

            Answer:
"""

response = llm_model(prompt, params)
print(f"prompt : {prompt} \n")
display(Markdown(response.content))

prompt : Classify the following sentence as true or false:
                'The Eiffel Tower is located in Berlin.'

            Answer:
 



False

In [31]:
# We are creating two prompts one for to classify a movie review as positive or negative and second one to translate an English phrase to Spanish.

movie_review_prompt = """Classify a movie review as positive or negative:

                        'F1 movie : This movie was a thrilling ride from start to finish! The racing scenes were intense, the acting was superb, and the story kept me on the edge of my seat. Highly recommended!'

                        Answer : 
                        
                        Comment :
"""

translate_prompt = """
                Translate the following sentence from English phrase to Spanish
                'Hi, How are doing gentlemen!!'

                Answer: 
                Comment : 

"""

prompts = {}
prompts["movie_review"] = movie_review_prompt
prompts["translate_lang"] = translate_prompt

for prompt_type, prompt in prompts.items():
    print(f"======={prompt_type.upper()} RESPONSE ======")
    response = llm_model(prompt, params)
    display(Markdown(response.content))

=======MOVIE_REVIEW RESPONSE ======


Answer: Positive

Comment: The review uses words like "thrilling," "intense," "superb," and "highly recommended," all indicating a positive experience.

=======TRANSLATE_LANG RESPONSE ======


**Answer:** ¡Hola! ¿Cómo están, caballeros?

**Comment:**

*   **¡Hola!** is the standard greeting for "Hi!"
*   **¿Cómo están?** is the formal "How are you?" in the plural. Using "están" is appropriate because you're addressing multiple people.
*   **caballeros** is the Spanish word for "gentlemen." It adds a polite and respectful tone.

#### 3. Few-shot prompt

Few-shot prompting extends the one-shot approach by providing multiple examples (typically 2-5) before asking the model to perform the task. These examples establish a clearer pattern and context, helping the model better understand the expected output format, style and reasoning. This technique is particularly effective for complex tasks where a single example might not convey all the nuances.

In [32]:
params = {
    "max_tokens" : 10,
}

prompt = """Here are few examples for classifying emotions in statements: 

        Statement: 'I just won my first marathon!'
        Emotion: Joy

        Statement: 'I can't believe I lost my keys again.'
        Emotion: Frustation

        Statement: 'My best friend is moving to another country.'
        Emotion: Sadness

        Now, classify the emotion in the following statement:
        Statement: 'That movie was so scary I had to cover my eyes.'

"""

response = llm_model(prompt, params)
print(f"prompt : {prompt} \n")
display(Markdown(response.content))

prompt : Here are few examples for classifying emotions in statements: 

        Statement: 'I just won my first marathon!'
        Emotion: Joy

        Statement: 'I can't believe I lost my keys again.'
        Emotion: Frustation

        Statement: 'My best friend is moving to another country.'
        Emotion: Sadness

        Now, classify the emotion in the following statement:
        Statement: 'That movie was so scary I had to cover my eyes.'

 



Emotion: Fear

#### 4. Chain-of-thought (CoT) prompt

Chain-of-thought (CoT) prompting encourages the model to break down complex problems into step-by-step reasoning before arriving at a final answer. By explicitly showing or requesting intermediate steps, this technique improves the model's problem-solving abilities and reducess errors in tasks requiring multi-step reasoning. CoT is particularly effective for mathematical problems, logical reasoning, and complex decision-making tasks.

In [33]:
params = {
    "max_new_tokens": 512, 
    "temperature": 0.5,
}

prompt = """Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples.
            How many apples are there now?'

    Break down each step of your calculation

"""

response = llm_model(prompt, params)
print(f"prompt: {prompt} \n")
display(Markdown(response.content))

prompt: Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples.
            How many apples are there now?'

    Break down each step of your calculation

 



Here's the breakdown of the calculation:

1. **Start:** The store begins with 22 apples.

2. **Sold:** They sell 15 apples, so we subtract that from the starting amount: 22 - 15 = 7 apples.

3. **Delivery:** They receive a delivery of 8 new apples, so we add that to the remaining amount: 7 + 8 = 15 apples.

**Therefore, the store now has 15 apples.**

In [40]:
params = {
    "max_new_tokens": 512, 
    "temperature": 0.5,
}

prompt = """Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples.
            How many apples are there now?'

            Answer: 
"""

response = llm_model(prompt, params)
print(f"prompt: {prompt} \n")
display(Markdown(response.content))

prompt: Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples.
            How many apples are there now?'

            Answer: 
 



Answer: 15

In [42]:
# params
params = {
    "max_tokens" : 256
}

# Prompt for decision-making process
decision_making_prompt = """ Consider the situation: 'Whether a student should study tonight
or go to a movie with friends, considering their upcoming test in two days.
Can you summarize the answer in 250 words and provide your decision'

Break down your reason:

"""
response = llm_model(decision_making_prompt, params)
print(f"prompt : {decision_making_prompt}")
display(Markdown(response.content))

prompt :  Consider the situation: 'Whether a student should study tonight
or go to a movie with friends, considering their upcoming test in two days.
Can you summarize the answer in 250 words and provide your decision'

Break down your reason:




Okay, here's a breakdown and summary of the decision:

**Reasoning Breakdown:**

*   **Academic Priority:** The upcoming test in two days is a significant factor. Academic performance is crucial for long-term goals.
*   **Time Management:** Two days provides some time for both studying and leisure. The key is to allocate time effectively.
*   **Study Effectiveness:** Cramming the night before a test is often less effective than spaced repetition. A focused study session tonight, followed by a lighter review tomorrow, might be optimal.
*   **Social Well-being:** Social interaction is important for mental health and can reduce stress. Completely isolating oneself before a test can be counterproductive.
*   **Opportunity Cost:** Choosing the movie means sacrificing study time. Choosing to study means missing out on social time with friends.

**Decision:**

The student should **study tonight, but with a compromise.**

**Summary (within 250 words):**

The student faces a classic dilemma: academics versus social life. While the upcoming test is important, completely sacrificing social interaction isn't ideal. The best approach is a balanced one. The student should dedicate a focused block of time (e.g., 2-3 hours

#### 6. Self-consistency

Self-consistency is an advanced technique in which the model generates multiple independent solutions or answers to the same problem, then evaluates these different approaches to determine most consistent or reliable result. This method enhances accuracy by leverarging the model's ability to approach problems from different angles and identify the most robust solution through comparison and verification.

In [46]:
params = {
    'max_tokens': 512,
}

prompt = """ When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

Provide three independent calculations and explanations, then determine the most consistent result.

"""

response = llm_model(prompt, params)
print(f"prompt: {prompt} \n")
display(Markdown(response.content))

prompt:  When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

Provide three independent calculations and explanations, then determine the most consistent result.

 



Here are three independent calculations to determine your sister's age, along with explanations, and a final determination:

**Calculation 1: Age Difference**

*   **Explanation:** The age difference between you and your sister remains constant throughout your lives.
*   **Calculation:**
    *   When you were 6, your sister was 6 / 2 = 3 years old.
    *   The age difference is 6 - 3 = 3 years.
    *   Therefore, your sister is always 3 years younger than you.
    *   Now that you are 70, your sister is 70 - 3 = 67 years old.

**Calculation 2: Relative Age Increase**

*   **Explanation:** We can track how many years have passed since the initial age difference was established.
*   **Calculation:**
    *   When you were 6, your sister was 3.
    *   Years passed: 70 - 6 = 64 years.
    *   Sister's age now: 3 + 64 = 67 years old.

**Calculation 3: Ratio Method**

*   **Explanation:** While the ratio of your ages changes, the *difference* remains constant. We can use the initial ratio to confirm the age difference.
*   **Calculation:**
    *   When you were 6, the ratio of your age to your sister's age was 2:1.
    *   This confirms the age difference of 3 years (6 - 3 = 3).
    *   Since you are now 70, your sister is 70 - 3 = 67 years old.

**Determination:**

All three calculations consistently result in your sister being **67 years old**. Therefore, this is the most consistent and likely correct answer.

# Now, we will explore LCEL Approach:

Five steps that is common while building application using LCEL approach:

1. Define the content or problem to be addressed.
2. Create a template with variables for dynamic content.
3. Convert the template into a LangChain PromptTemplate.
4. Build a chain using the pipe operator | to connect.
    1. Input Variables
    2. The prompt template
    3. The LLM
    4. An output parser

5. Invoke the chain with specific inputs to generate results.


In [ ]:
model_id = "gemini-2.0-flash"

default_params = {
    "temperature" : 0,
    "max_tokens" : None, 
    "timeout" : None, 
    "max_retries" : 2
}

llm = ChatGoogleGenerativeAI(
    model = model_id,
    temperature = default_params["temperature"],
    max_tokens = default_params["max_tokens"],
    timeout = default_params["timeout"],
    max_retries = default_params["max_retries"]
)

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x122574d70>, default_metadata=(), model_kwargs={})

In [65]:
template = """Tell me a {adjective} joke about {content}."""
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.')

In [66]:
# now, let's take a look how the prompt has been formatted
prompt.format(adjective = "funny", content = "chicken")

'Tell me a funny joke about chicken.'

To ensure consistent formatting of the prompts, we will define a helper function format_prompt. This function takes a dictionary of variables and applies them to our prompt template. 

In [67]:
# Define a function to ensure proper formatting
def format_prompt(variables):
    return prompt.format(**variables)

In [69]:
# The following code builds a chain using LCEL(LangChain Expression Language). This
# chain connects components using the pipe operator (|) to create a processing flow.
# The chain takes input variables, passes them through the prompt template, sends the 
# formatted prompt to the LLM, and uses a string output parser to return the final responses.
from langchain_core.runnables import RunnableLambda

joke_chain = (
    RunnableLambda(format_prompt)
    | llm
    | StrOutputParser()
)

response = joke_chain.invoke({"adjective":"sad", "content":"chicken"})
display(Markdown(response))

Why did the chicken cross the playground?

To get to the other slide.

...But he was too late. They'd already closed it down because of the avian flu. Now he's just standing there, staring at the empty slide, wondering what could have been.

In [70]:
# We can now just simply replace the varibles accordingly for other situation
response = joke_chain.invoke({"adjective":"funny", "content":"human being"})
display(Markdown(response))

Why did the human cross the road?

Because they saw something shiny on the other side and completely forgot what they were doing in the first place. Then they got distracted by a squirrel.

#### 1. Text Summarization

In [71]:
template = """ Summarize the {content} in two sentence
"""

# Create the prompt template
prompt = PromptTemplate.from_template(template)

# helper function for prompt template
def format_template(variables):
    return prompt.format(**variables)

In [74]:
content = """
    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. 
    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. 
    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. 
    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. 
    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""

summary_chain = (
    RunnableLambda(format_template)
    | llm
    | StrOutputParser()
)

summary = summary_chain.invoke({"content":content})
display(Markdown(summary))

Rapid technological advancements like AI, machine learning, and IoT are revolutionizing industries like healthcare, education, and transportation. These innovations are enhancing productivity, improving accessibility, and fostering a more interconnected and informed global society.

#### 2. Question Answering

In [ ]:
content = """
    The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets. 
    The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid. 
    The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

question = "Which planets in the solar system are rocky and solid?"

template = """
    Answer the {question} based on the {content}.
    Respond "Unsure about answer" if not sure about the answer.


    Answer:
    
"""

prompt = PromptTemplate.from_template(template)
